<a href="https://colab.research.google.com/github/sreelekaa/news-fake-detection/blob/main/News_Fake_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas


In [ ]:
import kagglehub
emineyetm_fake_news_detection_datasets_path = kagglehub.dataset_download('emineyetm/fake-news-detection-datasets')
clmentbisaillon_fake_and_real_news_dataset_path = kagglehub.dataset_download('clmentbisaillon/fake-and-real-news-dataset')

print('Data source import complete.')


Data source import complete.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
# Load both fake and real news datasets
fake = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/Fake.csv")
real = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/True.csv")

# Add labels
fake['label'] = 0  # 0 = Fake
real['label'] = 1  # 1 = Real

# Combine the datasets
data = pd.concat([fake, real], ignore_index=True)

# Show the first few rows
data.head()


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [ ]:
# Check label distribution
print(data['label'].value_counts())

# Check for missing values
print(data.isnull().sum())


label
0    23481
1    21417
Name: count, dtype: int64
title      0
text       0
subject    0
date       0
label      0
dtype: int64


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation and non-alphabetic chars
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Remove stopwords and stem
    cleaned = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(cleaned)


In [ ]:
data['cleaned_text'] = data['text'].apply(clean_text)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


In [ ]:
# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the cleaned text
X = vectorizer.fit_transform(data['cleaned_text']).toarray()

# Target labels
y = data['label'].values


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

lr_preds = lr_model.predict(X_test)

print("🔹 Logistic Regression Results:")
print("Accuracy:", accuracy_score(y_test, lr_preds))
print(classification_report(y_test, lr_preds))


🔹 Logistic Regression Results:
Accuracy: 0.984521158129176
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4733
           1       0.98      0.99      0.98      4247

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



In [ ]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

nb_preds = nb_model.predict(X_test)

print("🔹 Naive Bayes Results:")
print("Accuracy:", accuracy_score(y_test, nb_preds))
print(classification_report(y_test, nb_preds))


🔹 Naive Bayes Results:
Accuracy: 0.9204899777282851
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      4733
           1       0.93      0.90      0.91      4247

    accuracy                           0.92      8980
   macro avg       0.92      0.92      0.92      8980
weighted avg       0.92      0.92      0.92      8980



In [ ]:
# Save the TF-IDF vectorizer separately
with open("tfidf_vectorizer.pkl", "wb") as vec_file:
    pickle.dump(vectorizer, vec_file)


In [ ]:
import pickle
model_data = pickle.load(open("fake_news_model.pkl", "rb"))
model = model_data["model"]
vectorizer = model_data["vectorizer"]


In [ ]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier

# Suppose 'data' is your training DataFrame with 'cleaned_text' and 'label'

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['cleaned_text'])
y = data['label']

model = PassiveAggressiveClassifier(max_iter=50)
model.fit(X, y)

PassiveAggressiveClassifier(max_iter=50)

In [ ]:
# Save both model and vectorizer together
with open('fake_news_model.pkl', 'wb') as f:
    pickle.dump({'model': model, 'vectorizer': vectorizer}, f)

In [ ]:
app_code = """
import streamlit as st
import pickle
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = nltk.word_tokenize(text)
    cleaned = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(cleaned)

# Load model and vectorizer
loaded = pickle.load(open("fake_news_model.pkl", "rb"))

if isinstance(loaded, dict):
    model = loaded.get('model')
    vectorizer = loaded.get('vectorizer')
else:
    model = loaded
    vectorizer = pickle.load(open("tfidf_vectorizer.pkl", "rb"))

st.title("📰 Fake News Detector")

user_input = st.text_area("Enter a news article...")

if st.button("Predict"):
    if user_input.strip() == "":
        st.warning("Please enter some text to classify.")
    else:
        cleaned_input = clean_text(user_input)
        vector_input = vectorizer.transform([cleaned_input]).toarray()
        prediction = model.predict(vector_input)

        if prediction[0] == 0:
            st.error("🛑 This news article is likely **FAKE**.")
        else:
            st.success("✅ This news article appears to be **REAL**.")

"""

with open("app.py", "w") as f:
    f.write(app_code)


In [ ]:
!pip install streamlit pyngrok


In [ ]:
!ngrok config add-authtoken 2xV1j3h8eeC78pmVtnvQHMDy2va_6p36mUxeXfDwPh5xBVt57


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install --upgrade pyngrok


In [ ]:
from pyngrok import ngrok

# Start Streamlit app in background
get_ipython().system_raw('streamlit run app.py &')

# Open a tunnel specifying protocol 'http' and port 8501
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app URL: {public_url}")


Streamlit app URL: NgrokTunnel: "https://1766-34-169-203-61.ngrok-free.app" -> "http://localhost:8501"
